In [18]:
using DataFrames, Feather

In [19]:
@enum FailureType begin
    # General ones
    Unknown
    InferredFailure
    PrintingChange
    TestAmbiguities
    NewAmbiguity
    ApproxError
    DocTest
    NewExceptionType
    DownloadError
    UnexpectedPass
    BadBoy
    SyntaxError
    VisualRegression
    MissingDep
    BuildError
    UnsatReq
    
    # Specific
    InferenceStackOverFlow
    GetIndexExpr
    IRTools
    LLVMModuleIR
    BoundsErrorCompiler
    Cassette
    GPUErrors
    TypeSubtract
    OldRevise
    Grisu
    Formatting
    AssertTypeEqual
    AssertOpaqueType
    AssertJlTypesEqual
end

In [20]:
if !isdir("data")
    run(`tar -xvf data.tar.xz`)
end
primary  = Feather.read("data/primary.feather");
against = Feather.read("data/against.feather");

In [21]:
package_results = join(primary, against,
     on=:uuid, kind=:left, makeunique=true, indicator=:source);

In [22]:
changed_tests = filter(test->test.source == "both" &&
                 test.status != test.status_1, package_results)
fails = changed_tests[in.(changed_tests[!, :status], ((":fail", ":kill"),)), :];
sort!(fails , :name);
insert!(fails, size(fails, 2), fill(Unknown, size(fails, 1)), :why);

In [23]:
query(fails, s) = filter(row -> occursin(s, row[:log]), fails)    

function update_reason!(fails, needle, why)
    idxs = findall(row -> occursin(needle, row), fails.log)
    fails.why[idxs] .= why
    return fails
end

total_unknown(fails) = count(x -> x.why == Unknown, eachrow(fails))

total_unknown (generic function with 1 method)

In [24]:
# Generic ones
update_reason!(fails, "Test.detect_ambiguities", TestAmbiguities)
update_reason!(fails, "detect_ambiguities(", TestAmbiguities)

update_reason!(fails, "Expression: all_doctests()", DocTest)
update_reason!(fails, "Error: doctest failure in ", DocTest)

update_reason!(fails, "does not match inferred return type", InferredFailure);
update_reason!(fails, "Expression: isapprox", ApproxError)
update_reason!(fails, r"Expression: (\S*) ≈ (\S*)", ApproxError) 
update_reason!(fails, "Expression: ≈(", ApproxError)
    
update_reason!(fails, "is ambiguous. Candidates:", NewAmbiguity)
update_reason!(fails, " ambiguities found", NewAmbiguity)

update_reason!(fails, "Unsatisfiable requirements detected for package", UnsatReq)



update_reason!(fails, "Evaluated: \"", PrintingChange)
update_reason!(fails, "Log Test Failed at", PrintingChange)
update_reason!(fails, "Expression: occursin(r\"", PrintingChange)
update_reason!(fails, "Expression: startswith(", PrintingChange)
update_reason!(fails, r"Expression: (\N.*?) == ", PrintingChange)
update_reason!(fails, "Evaluated: occursin(", PrintingChange)
update_reason!(fails, "Evaluated: endswith(", PrintingChange)
update_reason!(fails, "- DIFF ------------------------", PrintingChange)
update_reason!(fails, "LoadError: syntax", SyntaxError)

update_reason!(fails, "Image did not match reference image", VisualRegression)

update_reason!(fails, "      Thrown: ", NewExceptionType)
update_reason!(fails, "The requested URL returned error", DownloadError)
update_reason!(fails, "gzip: stdin: not in gzip format", DownloadError)
update_reason!(fails, "Unexpected Pass", UnexpectedPass)

update_reason!(fails, "Error building ", BuildError)



# Specific ones for this release
update_reason!(fails, "Params not defined", IRTools);
update_reason!(fails, "Core.Compiler.IRCode(", IRTools);
update_reason!(fails, "/.julia/packages/IRTools/", IRTools);
update_reason!(fails, "Module IR does not contain specified entry function", LLVMModuleIR);
update_reason!(fails, "Internal error: encountered unexpected error in runtime:\nBoundsError", BoundsErrorCompiler
);
update_reason!(fails, "@ Cassette", Cassette)
update_reason!(fails, "Test.detect_ambiguities", TestAmbiguities)
update_reason!(fails, "unsupported or misplaced expression \"return\" in function", GPUErrors)
update_reason!(fails, "intersect_aside", InferenceStackOverFlow);

update_reason!(fails, "no method matching typesubtract", TypeSubtract);
update_reason!(fails, "MethodError(Core.Compiler.typesubtract", TypeSubtract);

update_reason!(fails, r"Package (\S.*) not found in current path:", MissingDep);



update_reason!(fails, "MethodError: no method matching Base.TOMLCache()", OldRevise)
update_reason!(fails, "UndefVarError: Grisu not defined", Grisu)
update_reason!(fails, "MethodError: no method matching parse(::String)", Formatting)
update_reason!(fails, "ERROR: LoadError: LoadError: MethodError: no method matching getindex(::Expr, ::Int64)", GetIndexExpr)


update_reason!(fails, "jl_types_equal at /workspace/srcdir/src/subtype.c:1987", AssertTypeEqual)
update_reason!(fails, "opaque types should have been handled specially", AssertOpaqueType)
update_reason!(fails, "Assertion `!jl_types_equal(mi->specTypes, type)", AssertJlTypesEqual)

# Baddies

# https://github.com/JuliaDiffusionBayes/DiffusionDefinition.jl/issues/11
update_reason!(fails, "DD.remove_curly(Vector{Float64}) == Array", BadBoy)
update_reason!(fails, "Unknown observable eltype", BadBoy)
update_reason!(fails, "ArgumentError: Invalid type: typename(SwingEq)", BadBoy)
update_reason!(fails, "/home/pkgeval/.julia/packages/Tricks/Omjx9/test/runtests.jl:89", BadBoy)
update_reason!(fails, "Judycon ~/.julia/packages/Judycon/Zc2IV/src/Judycon.jl:7", BadBoy)
update_reason!(fails, ": Error During Test at /home/pkgeval/.julia/packages/LOLTools/K3Wzy/test/loltools/championmasteryv4.jl:1", BadBoy)
update_reason!(fails, "/home/pkgeval/.julia/packages/Libtask/Zo6uM/test/ctask.jl:23", BadBoy)
update_reason!(fails, "~/.julia/packages/MonthlyDates/vCogo/src/MonthlyDates.jl:72", BadBoy)
update_reason!(fails, "type Method has no field ambig", BadBoy)
update_reason!(fails, "oadError: UndefVarError: LAST_SHOWN_LINE_INFOS not definedfatal: ", BadBoy)
update_reason!(fails, "/home/pkgeval/.julia/packages/CQLdriver/QjN09/deps/build.jl:1", BadBoy)
update_reason!(fails, "Cannot `convert` an object of type ForneyLab.Variable to an object of type Symbol", BadBoy)
update_reason!(fails, "est Failed at /home/pkgeval/.julia/packages/XPORTA/QaHQb/test/integration/filesystem.jl:41", BadBoy)
update_reason!(fails, "During Test at /home/pkgeval/.julia/packages/AlignedBinaryFormat/Nlf6L/test/runtests.jl:59", BadBoy)
update_reason!(fails, "ApproximateComputations/fS6jW/src/ASTReplacementApproximation.jl:121", BadBoy)
update_reason!(fails, "type OptimizationState has no field params", BadBoy)



nothing

In [35]:
issues_opened = [
    "AbstractNumbers", # https://github.com/SimonDanisch/AbstractNumbers.jl/issues/12
    "ArchGDAL", # https://github.com/yeesian/ArchGDAL.jl/issues/154
    "AutomotiveSimulator", #https://github.com/JuliaLang/julia/issues/38422
    "BilevelJuMP", # https://github.com/jump-dev/MathOptInterface.jl/pull/1197
    "DataTypesBasic", # https://github.com/JuliaLang/julia/issues/38423
    "Emoji_Entities", #https://github.com/JuliaString/Emoji_Entities.jl/issues/18
    "Contour", #https://github.com/JuliaGeometry/Contour.jl/issues/62
    "Continuables", # https://github.com/JuliaLang/julia/issues/38423
    "DoubleFloats", # https://github.com/JuliaMath/DoubleFloats.jl/issues/112
    "DevIL", # https://github.com/JuliaGL/DevIL.jl/pull/4,
    "EAGO", # https://github.com/PSORLab/EAGO.jl/issues/58
    "ExponentialUtilities", # https://github.com/JuliaLang/julia/issues/38426
    "Formatting", #https://github.com/JuliaIO/Formatting.jl/pull/88
    "Rotations", # https://github.com/JuliaGeometry/Rotations.jl/issues/133
    "Memento", # https://github.com/invenia/Memento.jl/pull/165
    "IonBase", # https://github.com/Roger-luo/IonBase.jl/pull/8
    "MLLabelUtils", # https://github.com/JuliaML/MLLabelUtils.jl/pull/41
    "GAP", # https://github.com/oscar-system/GAP.jl/issues/560
    "DataStructures", # https://github.com/JuliaCollections/DataStructures.jl/issues/703
    "DataTypesBasic", # https://github.com/JuliaLang/julia/issues/38423
]

likely_tol = [
    "AdvancedVI",
    "BlackBoxOptim",
    "CalibrationTests",
    "ConvexBodyProximityQueries",
    "DeterminantalPointProcesses",
    "FastTransforms",
    "FeynmanKacParticleFilters",
    "FiniteDifferences",
    "FunctionOperators",
    "GPLinearODEMaker",
    "GeneticVariation",
    "HMMBase"
];

In [26]:
total_unknown(fails)

100

In [36]:
filter(x -> x.why == Unknown, fails)

,julia,name,uuid
,String,String,String
1,"v""1.6.0-beta1-73f862b30f""",AWSCore,"UUID(""4f1ea46c-232b-54a6-9b17-cc2d0f3e6598"")"
2,"v""1.6.0-beta1-73f862b30f""",Autologistic,"UUID(""4d03fd4d-39a1-545a-9bf0-a2e993faffdf"")"
3,"v""1.6.0-beta1-73f862b30f""",AxisKeys,"UUID(""94b1ba4f-4ee9-5380-92f1-94cde586c3c5"")"
4,"v""1.6.0-beta1-73f862b30f""",BPFnative,"UUID(""b6338580-32ea-11e9-1791-33a79977d8c4"")"
5,"v""1.6.0-beta1-73f862b30f""",BlobTracking,"UUID(""738e0a8b-9125-4fc4-8d4f-3a5b1a21603b"")"
6,"v""1.6.0-beta1-73f862b30f""",Bukdu,"UUID(""3e78a19a-cc83-51d8-955b-515f39fd7955"")"
7,"v""1.6.0-beta1-73f862b30f""",CDDLib,"UUID(""3391f64e-dcde-5f30-b752-e11513730f60"")"
8,"v""1.6.0-beta1-73f862b30f""",CMPlot,"UUID(""79866cb8-e9c7-11e9-0ec0-4bf12ba13504"")"
9,"v""1.6.0-beta1-73f862b30f""",CalibrationTests,"UUID(""2818745e-0823-50c7-bc2d-405ac343d48b"")"


In [28]:
z = []
for i in instances(FailureType)
    n = count(row -> row.why == i, eachrow(fails))
    n == 0 || push!(z, (n, i))
end
sort!(z; rev=true)

20-element Array{Any,1}:
 (118, PrintingChange)
 (100, Unknown)
 (28, LLVMModuleIR)
 (21, DocTest)
 (16, UnsatReq)
 (14, ApproxError)
 (13, BuildError)
 (10, BadBoy)
 (9, Grisu)
 (6, IRTools)
 (6, NewExceptionType)
 (5, VisualRegression)
 (5, NewAmbiguity)
 (4, OldRevise)
 (4, MissingDep)
 (4, SyntaxError)
 (4, TestAmbiguities)
 (2, InferredFailure)
 (1, TypeSubtract)
 (1, Cassette)

In [29]:
d = filter(x -> x.why == Unknown, fails).name
d = filter(!in(issues_opened), d)
d = filter(!in(likely_tol), d)
@show length(d)
join(d, '\n') |> print

length(d) = 95
AWSCore
Autologistic
AxisKeys
BPFnative
BlobTracking
Bukdu
CDDLib
CMPlot
Cambrian
Circuitscape
ClimateBase
CloudWatchLogs
CombinedParsers
ComoniconGUI
DIVAnd
DataAssim
DensityRatioEstimation
DiffEqOperators
Dispersal
DynamicPolynomials
EmojiSymbols
Enzyme
ExaPF
ExtremeStats
FTPClient
FWFTables
FlashWeave
Flux
GEOTRACES
GalacticOptim
Gridap
GridapEmbedded
GridapODEs
GtkReactive
H3
HalfIntegerArrays
IPython
ImageInpainting
JLLWrappers
JetPackDSP
Jive
LOLTools
Laplacians
Libtask
LocalRegistry
MCAnalyzer
MCMCChain
MCMCChains
MGVI
Mads
MakieGallery
MathOptInterface
Millboard
Minesweeper
MultivariatePolynomials
MutableArithmetics
NCDatasets
NMFk
NamedPlus
NeXLSpectrum
NetCDF
NeuralNetDiffEq
NeuralPDE
OceanTurb
Oceananigans
OhMyREPL
OpenQuantumTools
OptimKit
Plotly
Polyhedra
PowerModels
PrincipalMomentAnalysisApp
ProbabilisticCircuits
Probably
ProximalOperators
QuantLib
Quante
RBNF
ReinforcementLearningZoo
RoME
ShallowWaters
SparseDiffTools
Stan
StochasticDiffEq
StructViews
Str